The jupyter notebook contains the function to run the model with some error handling.

In [3]:
import numpy as np
import copy

In [4]:
## function definition for single cycle of probability aggregation
def prob_agg2(current, prob):
   x = len(current)
   transmit = np.multiply(prob, np.tile(np.atleast_2d(current).T, (1,x)))
   current[:] = 1 - np.prod(1 - transmit, axis = 0) # P(A or B) = 1 - P(!A and !B)

In [ ]:
## function for the entire model
def model(presence, probabilities, num_cycles, prob_start=0, print_flag=False):
    # convert to numpy
    presence = np.array(presence)
    probabilities = np.array(probabilities)

    # get number of prob arrays
    num_dims = len(probabilities.shape)
    if num_dims == 3:
        num_probs = probabilities.shape[2]

    # Error handling
    if num_dims < 2 or num_dims > 3:
        print("ERROR: incorrect number of dimensions in probability array")
        return
    if probabilities.shape[0] != probabilities.shape[1]:
        print("ERROR: Probabilities Matrix not square")
        return
    if presence.shape[0] != probabilities.shape[0]:
        print("ERROR: Presence and Probabilities are not the same size")
        return
    if np.max(probabilities) > 1 or np.max(probabilities) < 0:
        print("ERROR: Invalid Probability value")
        return
    if len(set(presence) - set([0,1])) > 0:
        print("ERROR: Presence is not binary")
        return
    if num_cycles < 1:
        print("ERROR: Number of cycles cannot be less than 1")
        return
    if num_dims == 3 and prob_start >= num_probs:
        print("ERROR: Probability start index is greater than the number of probabilities")
        return

    # start state
    current_state = copy.deepcopy(presence).astype(np.float64)
    if print_flag:
        print(presence) # intial

    # run model - same probabilities
    if num_dims == 2:
        # fill diagonal
        np.fill_diagonal(probabilities, 1)

        for i in range(num_cycles):
            prob_agg2(current_state, probabilities)
            if print_flag:
                print(current_state) # print progress

    # run model - different probabilities
    if num_dims == 3:
        # fill diagonal
        for i in range(num_probs):
            np.fill_diagonal(probabilities[:,:,i], 1)

        curr_prob = prob_start
        for i in range(num_cycles):
            prob_agg2(current_state, probabilities[:,:,curr_prob])
            if print_flag:
                print(current_state) # print progress
            curr_prob += 1
            if curr_prob == num_probs:
                curr_prob = 0

    return current_state

In [24]:
## Example Usage
# intial presence of BMSB in each city
bmsb_presence = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

# probability of transit between each city. Dimension 1 (rows) is origin. Dimension 2 (columns) is destination. Dimension 3 (layers) is used for varying probabilities by time step
# The function automatically distinguish 2d and 3d probability arrays
# generate random between 0 and 0.05
probabilities = np.random.rand(20,20,12) / 50

model(bmsb_presence, probabilities, 30, prob_start=10, print_flag=True)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1.00000000e+00 1.00947624e-02 5.22269267e-03 1.65955832e-02
 7.30502532e-03 5.48400294e-03 1.39858726e-02 1.77204706e-04
 6.32992097e-03 1.66024364e-02 7.87813522e-04 9.62935146e-03
 1.76417700e-02 4.15833343e-04 2.08714840e-03 1.36656727e-02
 6.52315846e-03 5.66287934e-03 4.75611248e-03 6.50503972e-03]
[1.         0.02909497 0.02429115 0.02947828 0.01027362 0.01552115
 0.03390038 0.0138173  0.02096754 0.02327276 0.00280413 0.02346219
 0.03665488 0.00375792 0.0230861  0.0278819  0.01855725 0.01642822
 0.0111613  0.02071818]
[1.         0.03550225 0.03335614 0.03443655 0.03039026 0.01983712
 0.03980683 0.02991488 0.04347179 0.03919386 0.02589046 0.03936436
 0.04945399 0.01947291 0.02876304 0.05011883 0.03434943 0.02464441
 0.01900984 0.02748724]
[1.         0.04921543 0.04715296 0.04652681 0.05141993 0.03170441
 0.0563639  0.04310941 0.06278097 0.05431885 0.03458247 0.052606
 0.06825734 0.04181073 0.03736656 0.06766871 0.05593522 0.04269692
 0.

array([1.        , 0.9329036 , 0.92484124, 0.91173405, 0.920327  ,
       0.89357516, 0.92400267, 0.91924581, 0.9304486 , 0.91105398,
       0.92181003, 0.9238655 , 0.92941575, 0.9289251 , 0.91602343,
       0.92443048, 0.91743281, 0.91996005, 0.92287598, 0.93020751])